# Analyse de données textuelles avec python

## Utilisation du machine learning

Essayons de construire un premier classifieur pour les emails

On utilise scikit-learn et un modèle du type SVM

On va essayer de prédire si un SMS est un spam ou non

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

On récupère les données se trouvant dans SMSSpamCollection

In [2]:
import pandas as pd
# recuperation des données
data_sms = pd.read_table("./data/SMSSpamCollection", names=["label", "message"])
# statistique des labels
data_sms["label"].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

On commence par transformer notre variable à prédire en variable binaire.

In [3]:
from sklearn.preprocessing import LabelEncoder
encode_y = LabelEncoder()
y = encode_y.fit_transform(data_sms["label"])

On divise les données apprentissage / test en utilisant la fonction train_test_split de sklearn

In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data_sms["message"], y, test_size=0.2)

On utilise une méthode classique de traitement de données textuelles<br>
TF-IDF term frequency-inverse document frequency<br>
Il s'agit d'une méthode de feature extraction qui permet d'extraire des fréquences à partir de textes<br>
TF fréquence du mot dans le texte<br>
IDF importance du mot dans le corpus complet<br>

On crée un objet puis on fait un fit_transform sur les données d'apprentissage et un transform sur les données test

In [5]:
# on transforme en matrice creuse d’occurrence des mots (on transforme x_train
# et on applique à x_test la transformation)
from sklearn.feature_extraction.text import TfidfVectorizer

trans_vect = TfidfVectorizer()

x_train_trans = trans_vect.fit_transform(x_train)
x_test_trans = trans_vect.transform(x_test)

On crée un modèle de classification, ici SVM

In [6]:
from sklearn.svm import SVC
modele_svm = SVC()
modele_svm.fit(x_train_trans, y_train)

/home/mathieu/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

On va représenter les différents indices de qualité avec sklearn

In [7]:
# on vérifie la qualité du modèle sur les données de vaildation
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
print("Accuracy pour SVC :", accuracy_score(y_test, modele_svm.predict(x_test_trans)))

Accuracy pour SVC : 0.873542600896861


Quelle est l'efficacité de ce classifieur ?

Essayez avec un random forest

In [8]:
from sklearn.ensemble import RandomForestClassifier
modele_rf = RandomForestClassifier()
modele_rf.fit(x_train_trans, y_train)
print("Accuracy pour RF :", accuracy_score(y_test, modele_rf.predict(x_test_trans)))

Accuracy pour RF : 0.9748878923766816


/home/mathieu/miniconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


...

Construisez un filtre anti-spam à partir de ces modèles

In [18]:
from sklearn.pipeline import make_pipeline
import numpy as np
# on construit un pipeline de traitement
pipe_text = make_pipeline(TfidfVectorizer(), RandomForestClassifier())
# on l’ajuste à toutes les données
# sachant qu’on a déjà validé le modèle
pipe_text.fit(x_train, y_train)

/home/mathieu/miniconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [10]:
print("Accuracy pour Pipeline :", accuracy_score(y_test, pipe_text.predict(x_test)))

Accuracy pour Pipeline : 0.9739910313901345


In [11]:
# on crée une fonction de filtre qui affiche un message
def message_filter(message) :
    arr_mess = np.array([message])
    result = encode_y.inverse_transform(pipe_text.predict(arr_mess))[0]
    print("The message you received is a :", result)

In [19]:
# on appelle la fonction message_filter avec un nouveau message
message_filter("WINNER click the WAP link in the next txt message")

The message you received is a : spam


In [13]:
message_filter("Hello, how are u?")

The message you received is a : ham
